In [17]:
import json, hmac, hashlib, time, requests, base64
from requests.auth import AuthBase

# Create custom authentication for Exchange
class GdaxAuth(AuthBase):
    def __init__(self, api_key, secret_key, passphrase):
        self.api_key = api_key
        self.secret_key = secret_key
        self.passphrase = passphrase

    def __call__(self, request):
        timestamp = str(time.time())
        message = timestamp + request.method + request.path_url + (request.body or '')
        message = message.encode('ascii')
        hmac_key = base64.b64decode(self.secret_key)
        signature = hmac.new(hmac_key, message, hashlib.sha256)
        signature_b64 = base64.b64encode(signature.digest()).decode('utf-8')

        request.headers.update({
            'CB-ACCESS-SIGN': signature_b64,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-ACCESS-PASSPHRASE': self.passphrase,
            'Content-Type': 'application/json'
        })
        return request
    



In [24]:
CLIENT_SECRETS = 'client_secrets.json'
client_secrets = json.load(open(CLIENT_SECRETS))
api_url = client_secrets["sandbox"]["api_url"]
API_KEY = client_secrets["sandbox"]["api_key"]
API_SECRET = client_secrets["sandbox"]["api_secret"]
API_PASS = client_secrets["sandbox"]["api_pass"]
auth = GdaxAuth(API_KEY, API_SECRET, API_PASS)

# Get accounts
r = requests.get(api_url + '/products/BTC-USD/stats', auth=auth)

#r = requests.post(api_url + 'orders', json=order, auth=auth)
print (r.json())

{'open': '10990.23000000', 'high': '1000000.00000000', 'low': '1.00000000', 'volume': '9.99102425', 'last': '1000000.00000000', 'volume_30day': '61.36461233'}
